# Analyze a cross-correlation to produce station clock correction file

Prior to using this script, the quality of the correction should be visualized and confirmed using notebook `plotStationPairXcorr.ipynb`

This notebook will generate a csv file with dates and estimated clock corrections for a given station. Applying the correction to the original ASDF database will be done separately for the sake of safety, so that any changes to ASDF must be very deliberate and intentional.

In [ ]:
import os
import sys
import datetime

In [ ]:
import numpy as np
import scipy
import matplotlib.dates
import matplotlib.pyplot as plt
from dateutil import rrule
import pandas as pd

In [ ]:
import obspy

In [ ]:
package_root = os.path.abspath('../../..')
if package_root not in sys.path:
    sys.path.append(package_root)
from seismic.xcorqc.xcorr_station_clock_analysis import (XcorrPreprocessor, 
                                                         compute_estimated_clock_corrections,
                                                         plot_estimated_timeshift)

In [ ]:
SRC_FILE = "/g/data/ha3/am7399/shared/xcorr/7X/MA43_QIS/7X.MA43.AU.QIS.1.0-10.0.nc"
# SRC_FILE = "/g/data/ha3/am7399/shared/xcorr/7X/MA52_QIS/7X.MA52.AU.QIS.1.0-10.0.nc"
assert os.path.exists(SRC_FILE), "File not found!"

In [ ]:
_, basename = os.path.split(SRC_FILE)
name_parts = basename.split('.')
NETCODE = name_parts[0]
STATCODE = name_parts[1]
print("Inferred target station code: {}.{}".format(NETCODE, STATCODE))
FULL_CODE = '.'.join([NETCODE, STATCODE])

In [ ]:
TIME_WINDOW = 300 # +/-
SNR_THRESHOLD = 6

In [ ]:
xcorr_preproc = XcorrPreprocessor(SRC_FILE, TIME_WINDOW, SNR_THRESHOLD)

In [ ]:
PCF_CUTOFF_THRESHOLD = 0.5
rcf_corrected, correction, row_rcf_crosscorr = \
    compute_estimated_clock_corrections(xcorr_preproc.rcf, xcorr_preproc.snr_mask,
                                        xcorr_preproc.ccf_masked, xcorr_preproc.lag,
                                        PCF_CUTOFF_THRESHOLD)

In [ ]:
# Plot the data for visual check
plt.figure(figsize=(16,16))
plt.subplot(1, 2, 1)
ax = plt.gca()
plot_estimated_timeshift(ax, xcorr_preproc.lag, xcorr_preproc.start_times, correction)
date_formatter = matplotlib.dates.DateFormatter("%Y-%m-%d")
date_locator = matplotlib.dates.WeekdayLocator(byweekday=rrule.SU)
ax.yaxis.set_major_formatter(date_formatter)
ax.yaxis.set_major_locator(date_locator)
ytl = ax.get_yticklabels()
for _ytl in ytl:
    _ytl.set_visible(True)
ax.set_ylabel('Days')
ax.set_title('Computed clock corrections')

plt.subplot(1, 2, 2)
ax = plt.gca()
plot_estimated_timeshift(ax, xcorr_preproc.lag, xcorr_preproc.start_times, correction, 
                         row_rcf_crosscorr=row_rcf_crosscorr)
ax.yaxis.set_major_formatter(date_formatter)
ax.yaxis.set_major_locator(date_locator)
ax.set_title('RCF * CCF used for corrections')

plt.suptitle('GPS Clock Corrections for {}'.format(xcorr_preproc.src_file), fontsize=16, y=0.92)
plt.show()

## Segment the corrections time series into coherent groups

In [ ]:
flt_times = np.array([float(v) for v in xcorr_preproc.start_times])

In [ ]:
from sklearn.cluster import dbscan
from scipy import signal
import math

In [ ]:
sec_per_week = 7*24*3600

tuned_coeffs = {
    '7X.MA43': (1, 1, 20),
    '7X.MA52': (1, 5, 15)
}


def temporalDist2D(p0, p1):
    return math.sqrt((tuned_coeffs[FULL_CODE][0] * (p1[0] - p0[0]))**2 +
                     (tuned_coeffs[FULL_CODE][1] * sec_per_week*(p1[1] - p0[1]))**2)

def calendarDist(p0, p1):
    return abs(p1[0] - p0[0])


assert not np.any(np.isnan(flt_times))
nan_mask = np.isnan(correction)
times_nonan = flt_times[~nan_mask]
correction_nonan = correction[~nan_mask]
data = np.column_stack((times_nonan, correction_nonan))

ind, ids = dbscan(data, eps=2*sec_per_week, min_samples=7, metric=temporalDist2D)

In [ ]:
np_times = np.array([datetime.datetime.utcfromtimestamp(v) for v in xcorr_preproc.start_times]).astype('datetime64[ms]')
np_times = np_times[~nan_mask]

## Plot clusters based on sample positions in time

In [ ]:
plt.figure(figsize=(16,9))

plt.plot(np_times, correction_nonan, 'x', color="#808080")
for i in range(max(ids) + 1):
    mask_group = (ids == i)
    plt.plot(np_times[mask_group], correction_nonan[mask_group], 'o-', color='C{}'.format(i),
             markersize=6, fillstyle='none', alpha=0.7)

time_formatter = matplotlib.dates.DateFormatter("%Y-%m-%d")
plt.axes().xaxis.set_major_formatter(time_formatter)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.grid(':', color="#808080", zorder=0, alpha=0.5)
plt.xlabel('Day', fontsize=14)
plt.ylabel('Correction (sec)', fontsize=14)
plt.tight_layout()
plt.gcf().autofmt_xdate()
plt.title("Station {} zeroth order corrections groups".format(FULL_CODE), fontsize=20)
plt.show()

#### Note how ignoring the slope results in last group not being split into two. Next we add the slope to the distance metric.

## Add slope metric

In [ ]:
grad = np.gradient(correction_nonan, times_nonan, edge_order=1)
grad_med5 = signal.medfilt(grad, 5)

In [ ]:
slope = grad_med5

def temporalDist2DSlope(p0, p1):
    return math.sqrt((tuned_coeffs[FULL_CODE][0] * (p1[0] - p0[0]))**2 + 
                     (tuned_coeffs[FULL_CODE][1] * sec_per_week*(p1[1] - p0[1]))**2 +
                     (tuned_coeffs[FULL_CODE][2] * sec_per_week*sec_per_week*(p1[2] - p0[2]))**2)

assert not np.any(np.isnan(flt_times))
data = np.column_stack((times_nonan, correction_nonan, slope))
# print(data)

ind2, ids2 = dbscan(data, eps=2*sec_per_week, min_samples=7, metric=temporalDist2DSlope)

In [ ]:
plt.figure(figsize=(16,9))

plt.plot(np_times, correction_nonan, 'x', color="#808080")
for i in range(max(ids2) + 1):
    mask_group = (ids2 == i)
    plt.plot(np_times[mask_group], correction_nonan[mask_group], 'o-', color='C{}'.format(i),
             markersize=5, fillstyle='none')

time_formatter = matplotlib.dates.DateFormatter("%Y-%m-%d")
ylims = plt.ylim()
plt.axes().xaxis.set_major_formatter(time_formatter)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.grid(':', color="#808080", zorder=0, alpha=0.5)
plt.xlabel('Day', fontsize=14)
plt.ylabel('Correction (sec)', fontsize=14)
plt.tight_layout()
plt.gcf().autofmt_xdate()
plt.title("Station {} first order corrections groups".format(FULL_CODE), fontsize=20)
plt.show()

## With successful segmentation, we perform regression for each cluster

In [ ]:
len(times_nonan), len(np_times)

In [ ]:
num_segments = len(set(ids2[ids2 != -1]))
print(num_segments)

In [ ]:
# Dict of regressions, keyed by group ID
regressions = {}

# Corrections from regression function fit
correction_fit = np.zeros_like(correction_nonan)
correction_fit[:] = np.nan

In [ ]:
for i in range(num_segments):
    mask_group = (ids2 == i)
    # Perform regression
    x = times_nonan[mask_group]
    y = correction_nonan[mask_group]
    r = np.polyfit(x, y, 1)
    regressions[i] = r
    # Compute fitted values
    correction_fit[mask_group] = np.polyval(r, x)

### Replot with fitted line

In [ ]:
plt.figure(figsize=(16,9))

# plt.plot(np_times, correction_nonan, 'x', color="#808080")
for i in range(num_segments):
    mask_group = (ids2 == i)
    plt.plot(np_times[mask_group], correction_nonan[mask_group], 'o', color='C{}'.format(i),
             markersize=5, fillstyle='none')
    plt.plot(np_times[mask_group], correction_fit[mask_group], color='C{}'.format(i))

time_formatter = matplotlib.dates.DateFormatter("%Y-%m-%d")
plt.axes().xaxis.set_major_formatter(time_formatter)
plt.ylim(ylims)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.grid(':', color="#808080", zorder=0, alpha=0.5)
plt.xlabel('Day', fontsize=14)
plt.ylabel('Correction (sec)', fontsize=14)
plt.tight_layout()
plt.gcf().autofmt_xdate()
plt.title("Station {} corrections groups with regressions to sample times".format(FULL_CODE), fontsize=20)
plt.show()

In [ ]:
# Dict of daily spaced time values and computed correction, since source data time
# points might not be uniformly distributed. Keyed by group ID. These are the times
# at which we will output corrections.
regular_corrections = {}

In [ ]:
sec_per_day = 24*3600

for i in range(num_segments):
    mask_group = (ids2 == i)
    # Generate uniform daily times at which to compute corrections
    x = times_nonan[mask_group]
    timestamp_min = min(x)
    timestamp_max = max(x)
    num_days = np.round((timestamp_max - timestamp_min)/sec_per_day)
    lin_times = np.linspace(timestamp_min, timestamp_max, num_days + 1)
    lin_corrections = np.polyval(regressions[i], lin_times)
    regular_corrections[i] = {'times': lin_times, 'corrections': lin_corrections}

In [ ]:
# Replot to sanity check the final daily correction values
plt.figure(figsize=(16,9))

# plt.plot(np_times, correction_nonan, 'x', color="#808080")
for i in range(num_segments):
    mask_group = (ids2 == i)
    plt.plot(np_times[mask_group], correction_nonan[mask_group], 'o', color='#808080'.format(i),
             markersize=5, fillstyle='none')
    np_times_i = np.array([datetime.datetime.utcfromtimestamp(v) for v in 
                           regular_corrections[i]['times']]).astype('datetime64[ms]')
    plt.plot(np_times_i, regular_corrections[i]['corrections'], 'o', color='C{}'.format(i), fillstyle='none')

time_formatter = matplotlib.dates.DateFormatter("%Y-%m-%d")
plt.axes().xaxis.set_major_formatter(time_formatter)
plt.ylim(ylims)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.grid(':', color="#808080", zorder=0, alpha=0.5)
plt.xlabel('Day', fontsize=14)
plt.ylabel('Correction (sec)', fontsize=14)
plt.tight_layout()
plt.gcf().autofmt_xdate()
plt.title("Station {} corrections groups with regressions to daily samples".format(FULL_CODE), fontsize=20)
plt.show()

## Output regression results to csv file

Use tabular format for ease of use and interoperability, even though there will be some redundancy of information.

In [ ]:
data_blocks = []
for k in regular_corrections.keys():
    c = regular_corrections[k]
    # BEWARE: The 'corrections' array sign is negated there, since the correction
    # we have computed up to this point is actually the clock *error*. Subtraction
    # of an error is the same as addition of a correction of opposite sign.
    data_blocks.append(pd.DataFrame(np.column_stack([c['times'], -c['corrections']]), 
                                    columns=['timestamp', 'clock_correction']))
df = pd.concat(data_blocks)

In [ ]:
df['date'] = df['timestamp'].apply(obspy.UTCDateTime).apply(lambda x: x.date)

In [ ]:
df['net'] = NETCODE
df['sta'] = STATCODE

In [ ]:
df = df[['net', 'sta', 'date', 'clock_correction']]

In [ ]:
df[0:10]

In [ ]:
output_file = FULL_CODE + "_clock_correction.csv"

In [ ]:
df.to_csv(output_file, index=False)